In [1]:
from scrapy import spiders

In [6]:
class SemanticScholarSpider(spiders):
    name = 'semanticscholar'

    TITLE = "//h1[@data-selenium-selector='paper-detail-title']/text()" #DONE
    DATE = "//span[@data-selenium-selector='paper-year']/span/span/text()" #DONE
    AUTHORS = "//span[@data-selenium-selector='paper-meta-subhead']/span/span/a/span/span/text()" #DONE
    MORE_AUTHORS = "//span[@data-selenium-selector='paper-meta-subhead']/span/span/span/span[@class='more-authors-label']"
    ABSTRACT = "//meta[@name='description']/@content" #DONE
    REFERENCES = ''
    
    def __init__(self, start_urls, maximum_no=2000):
        self.start_urls = start_urls
        self.crawled = start_urls.copy()
        self.maximum_no = maximum_no
    
    def start_requests(self):
        urls = self.start_urls
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
            
    def parse(self, response):
        url = response.url
        id = url.split('/')[-1].split('_')[0]
        
        
START_URLS = [
    'https://www.semanticscholar.org/paper/The-Lottery-Ticket-Hypothesis%3A-Training-Pruned-Frankle-Carbin/f90720ed12e045ac84beb94c27271d6fb8ad48cf',
    'https://www.semanticscholar.org/paper/Attention-is-All-you-Need-Vaswani-Shazeer/204e3073870fae3d05bcbc2f6a8e263d9b72e776',
    'https://www.semanticscholar.org/paper/BERT%3A-Pre-training-of-Deep-Bidirectional-for-Devlin-Chang/df2b0e26d0599ce3e70df8a9da02e51594e0e992'   
]

TypeError: module.__init__() takes at most 2 arguments (3 given)